In [1]:
import numpy as np
import pandas as pd
ratings_filename='data/u.data'
all_ratings = pd.read_csv(ratings_filename,delimiter='\t',header=None,
                         names=['UserID','MovieID','Rating','Datatime'])
all_ratings['Datatime'] = pd.to_datetime(all_ratings['Datatime'],unit='s')  #时间格式转换
all_ratings[:5]

d:\dl\envs\gluon\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\dl\envs\gluon\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
d:\dl\envs\gluon\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
d:\dl\envs\gluon\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


,UserID,MovieID,Rating,Datatime
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16


In [2]:
#创建一个新特征 Favorable
all_ratings['Favorable'] = all_ratings['Rating'] > 3
all_ratings

,UserID,MovieID,Rating,Datatime,Favorable
0,196,242,3,1997-12-04 15:55:49,False
1,186,302,3,1998-04-04 19:22:22,False
2,22,377,1,1997-11-07 07:18:36,False
3,244,51,2,1997-11-27 05:02:03,False
4,166,346,1,1998-02-02 05:33:16,False
...,...,...,...,...,...
99995,880,476,3,1997-11-22 05:10:44,False
99996,716,204,5,1997-11-17 19:39:03,True
99997,276,1090,1,1997-09-20 22:49:55,False
99998,13,225,2,1997-12-17 22:52:36,False


In [3]:
#提取前200个数据作为训练集
ratings = all_ratings[all_ratings['UserID'].isin(range(200))]
favorable_ratings = ratings[ratings['Favorable']]   #新建一个数据集，只包括用户喜欢某部电源的数据行
#统计每个用户各喜欢哪些电影，按照UserID进行分组，并遍历每个用户看过的每一部电影
favorable_reviews_by_users = dict((k,frozenset(v.values)) for k,v in favorable_ratings.groupby('UserID')['MovieID'])
print(favorable_reviews_by_users)

{1: frozenset({1, 3, 6, 7, 9, 12, 13, 14, 15, 16, 18, 19, 20, 22, 23, 25, 28, 32, 33, 39, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 55, 56, 57, 58, 59, 60, 61, 64, 65, 66, 68, 72, 75, 76, 77, 79, 80, 81, 82, 84, 86, 87, 88, 89, 90, 91, 93, 95, 96, 98, 100, 106, 107, 108, 109, 111, 113, 114, 115, 119, 121, 123, 124, 127, 128, 129, 132, 133, 134, 135, 137, 144, 146, 150, 151, 152, 154, 156, 157, 160, 161, 162, 163, 165, 166, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 181, 182, 183, 184, 185, 186, 187, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 202, 203, 204, 206, 207, 208, 209, 210, 212, 214, 216, 221, 222, 223, 224, 227, 228, 229, 230, 234, 235, 236, 238, 239, 241, 242, 246, 248, 249, 250, 251, 253, 256, 257, 258, 265, 267, 268, 269, 270}), 2: frozenset({257, 1, 13, 14, 269, 272, 273, 275, 276, 277, 279, 25, 282, 283, 284, 285, 286, 292, 293, 295, 297, 299, 300, 301, 302, 303, 304, 50, 306, 310, 311, 313, 316, 100, 237, 111, 242, 251, 127, 255}), 3: frozenset({320, 321,

In [4]:
#再新建一个数据集，统计每部电影获得的喜欢数(点赞个数)
num_favorable_by_movie=ratings[['MovieID','Favorable']].groupby('MovieID').sum()
print(num_favorable_by_movie.sort_values('Favorable',ascending=False))

         Favorable
MovieID           
50             100
100             89
258             83
181             79
174             74
...            ...
677              0
1146             0
1145             0
695              0
1420             0

[1420 rows x 1 columns]


In [5]:
#初始化一个字典
frequent_itemsets={}
#最小点赞数
min_support=80
#为每一部电影生成只包含它自己的项目，检测它是否够频繁
frequent_itemsets[1]=dict((frozenset((movie_id,)),row['Favorable']) for movie_id,row in num_favorable_by_movie.iterrows() if row['Favorable'] > min_support)
print(frequent_itemsets)

{1: {frozenset({50}): 100, frozenset({100}): 89, frozenset({258}): 83}}


In [6]:
import sys
from collections import defaultdict
def find_frequent_itemsets(favorable_reviews_by_users,k_1_itemsets,min_support):
    counts=defaultdict(int)
    for user,reviews in favorable_reviews_by_users.items():
        for itemset in k_1_itemsets:
            if itemset.issubset(reviews):
                for other_reviewed_movie in reviews-itemset:
                    current_superset=itemset|frozenset((other_reviewed_movie,))
                    counts[current_superset]+=1
    print(counts)
    return dict([(itemset, frequency) for itemset,frequency in counts.items() if frequency >= min_support])
for k in range(2,20):
    cur_frequent_itemsets=find_frequent_itemsets(favorable_reviews_by_users,frequent_itemsets[k-1],min_support)
    frequent_itemsets[k]=cur_frequent_itemsets
    if len(cur_frequent_itemsets)==0:
        print("Did not find any frequent item")
    else:
        print("I found {} frequent of length{}".format(len(cur_frequent_itemsets),k))
        sys.stdout.flush()
del frequent_itemsets[1]
print(frequent_itemsets)

defaultdict(<class 'int'>, {frozenset({1, 50}): 50, frozenset({50, 3}): 3, frozenset({50, 6}): 4, frozenset({50, 7}): 47, frozenset({9, 50}): 35, frozenset({50, 12}): 31, frozenset({50, 13}): 20, frozenset({50, 14}): 23, frozenset({50, 15}): 29, frozenset({16, 50}): 3, frozenset({50, 18}): 2, frozenset({50, 19}): 6, frozenset({50, 20}): 6, frozenset({50, 22}): 29, frozenset({50, 23}): 20, frozenset({25, 50}): 26, frozenset({50, 28}): 31, frozenset({32, 50}): 13, frozenset({33, 50}): 8, frozenset({50, 39}): 6, frozenset({50, 42}): 15, frozenset({50, 43}): 1, frozenset({50, 44}): 9, frozenset({50, 45}): 5, frozenset({50, 46}): 2, frozenset({50, 47}): 9, frozenset({48, 50}): 16, frozenset({50, 51}): 7, frozenset({50, 52}): 10, frozenset({50, 55}): 14, frozenset({56, 50}): 46, frozenset({57, 50}): 5, frozenset({50, 58}): 12, frozenset({50, 59}): 12, frozenset({50, 60}): 6, frozenset({50, 61}): 9, frozenset({64, 50}): 41, frozenset({65, 50}): 12, frozenset({50, 66}): 12, frozenset({50, 68})

defaultdict(<class 'int'>, {frozenset({1, 50, 100}): 32, frozenset({50, 3, 100}): 2, frozenset({50, 100, 6}): 3, frozenset({50, 100, 7}): 35, frozenset({9, 50, 100}): 25, frozenset({50, 100, 12}): 25, frozenset({50, 100, 13}): 17, frozenset({50, 100, 14}): 17, frozenset({50, 100, 15}): 17, frozenset({16, 50, 100}): 3, frozenset({50, 18, 100}): 2, frozenset({50, 19, 100}): 6, frozenset({50, 100, 20}): 6, frozenset({50, 100, 22}): 20, frozenset({50, 100, 23}): 13, frozenset({25, 50, 100}): 17, frozenset({50, 100, 28}): 14, frozenset({32, 50, 100}): 12, frozenset({33, 50, 100}): 5, frozenset({50, 100, 39}): 5, frozenset({50, 100, 42}): 12, frozenset({50, 43, 100}): 1, frozenset({50, 100, 44}): 6, frozenset({50, 100, 45}): 5, frozenset({50, 100, 46}): 2, frozenset({50, 100, 47}): 7, frozenset({48, 50, 100}): 11, frozenset({50, 51, 100}): 5, frozenset({50, 100, 52}): 7, frozenset({50, 100, 55}): 11, frozenset({56, 50, 100}): 33, frozenset({57, 50, 100}): 4, frozenset({50, 100, 58}): 9, froz

In [7]:
#抽取关联规则
candidate_rules=[]
for itemset_length,itemset_counts in frequent_itemsets.items():
    for itemset in itemset_counts.keys():
        #遍历项集中的每一部电影，把它作为结论
        for conclusion in itemset:
            #项集中的其他电影作为前提
            premise=itemset - set((conclusion,))
            #用前提和结论组成备选规则
            candidate_rules.append((premise,conclusion))
print(candidate_rules[:5])
        
        

[(frozenset({100}), 50), (frozenset({50}), 100), (frozenset({258}), 50), (frozenset({50}), 258)]


In [12]:
#创建两个字典，用来存储规则正例和反例的次数
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
#遍历每个用户喜欢的电影数据
for user,reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise,conclusion = candidate_rule
        #判断用户是否喜欢前提中的电影
        if premise.issubset(reviews):
            #如果前提符合，看一下用户喜欢结论中的电影
            if conclusion in reviews:
                correct_counts[candidate_rule]+=1
            else:
                incorrect_counts[candidate_rule]+=1
#用正例次数除以前提条件出现的总次数，计算每条规则的置信度
rule_confidence={candidate_rule:correct_counts[candidate_rule]/float(correct_counts[candidate_rule]+incorrect_counts[candidate_rule]) for candidate_rule in candidate_rules}
#最小支持置信度
min_confidence = 0.5
#筛选出执行度大于0.5的
rule_confidence = {rule:confidence for rule,confidence in rule_confidence.items() if confidence > min_confidence}
print(len(rule_confidence))

3


In [13]:
from operator import itemgetter
#对置信度进行排序
sorted_confidence = sorted(rule_confidence.items(),key=itemgetter(1),reverse=True)
for index in range(len(sorted_confidence)):
    print("Rule #{0}".format(index+1))
    (premise,conclusion) = sorted_confidence[index][0]
    print("Rule: If a person reommends {0} they will also recommend {1}".format(premise,conclusion))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))

Rule #1
Rule: If a person reommends frozenset({100}) they will also recommend 50
 - Confidence: 0.618
Rule #2
Rule: If a person reommends frozenset({50}) they will also recommend 100
 - Confidence: 0.550
Rule #3
Rule: If a person reommends frozenset({258}) they will also recommend 50
 - Confidence: 0.518


In [14]:
movie_name_filename='data/u.item'
movie_name_data=pd.read_csv(movie_name_filename,delimiter='|',header=None,encoding='mac-roman')
movie_name_data.columns = ["MovieID", "Title", "Release Date", "Video Release", "IMDB", "<UNK>", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir",                            "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
def get_movie_name(movie_id):
    title_object = movie_name_data[movie_name_data["MovieID"] == movie_id]["Title"]
    title = title_object.values[0]
    return title
get_movie_name(4)
for index in range(len(sorted_confidence)):
    if rule_confidence[(premise, conclusion)]==1:
        print("Rule #{0}".format(index + 1))
        (premise, conclusion) = sorted_confidence[index][0]
        premise_names = ", ".join(get_movie_name(idx) for idx in premise)
        conclusion_name = get_movie_name(conclusion)
        print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
        print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
        print("")


In [15]:
#训练集前200名用户的打分数据，测试集就用剩下的数据。
#用于测试的数据，200条以后
test_dataset=all_ratings[~all_ratings['UserID'].isin(range(200))]
#提取用户喜欢的电影
test_favorable = test_dataset[test_dataset['Favorable']]
test_not_favorable = test_dataset[~test_dataset['Favorable']]
#统计每个用户喜欢哪些电影，按照UerID进行分组，并遍历每个用户看过的每一部电影
test_favorable_by_users = dict((k,frozenset(v.values)) for k,v in test_favorable.groupby('UserID')['MovieID'])
test_not_favorable_by_users = dict((k,frozenset(v.values)) for k,v in test_not_favorable.groupby('UserID')['MovieID'])

#创建两个字典，用来存储正例和反例的次数
correct_counts=defaultdict(int)
incorrect_counts=defaultdict(int)

for user,reviews in test_favorable_by_users.items():
    #遍历每条关联规则
    for candidate_rule in candidate_rules:
        premise,conclusio = candidate_rule
        #判断用户是否喜欢前提中的电影
        if premise.issubset(reviews):
            #如果前提符合，看一下用户是否喜欢结论中的电影
            if conclusion in reviews:
                correct_counts[candidate_rule]+=1
            else:
                incorrect_counts[candidate_rule]+=1

# 用正例次数除以前提条件出现的总次数，计算每条规则的置信度
test_confidence = {candidate_rule:correct_counts[candidate_rule]/float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
                   for candidate_rule in rule_confidence}

sorted_test_confidence = sorted(test_confidence.items(),key=itemgetter(1),reverse=True)
print(sorted_confidence[:5])
for index in range(len(sorted_confidence)):
    print("Rule #{0}".format(index+1))
    (premise,conclusion) = sorted_confidence[index][0]
    premise_names=','.join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Train Confidence: {0:.3f}".format(rule_confidence.get((premise, conclusion), -1)))
    print(" - Test Confidence: {0:.3f}".format(test_confidence.get((premise, conclusion), -1)))
    print("")



[((frozenset({100}), 50), 0.6179775280898876), ((frozenset({50}), 100), 0.55), ((frozenset({258}), 50), 0.5180722891566265)]
Rule #1
Rule: If a person recommends Fargo (1996) they will also recommend Star Wars (1977)
 - Train Confidence: 0.618
 - Test Confidence: 0.669

Rule #2
Rule: If a person recommends Star Wars (1977) they will also recommend Fargo (1996)
 - Train Confidence: 0.550
 - Test Confidence: 1.000

Rule #3
Rule: If a person recommends Contact (1997) they will also recommend Star Wars (1977)
 - Train Confidence: 0.518
 - Test Confidence: 0.632

